# Getting started

In [1]:
import pandas as pd
import numpy as np
import random

import akiFlagger

### Let's start off by creating some toy data

In [90]:
np.random.seed(0) #seed for reproducibility

#pick admission dates from Jan 1, 2020 to July 1, 2020 (6 month period) and generate time deltas from +- 5 days
date_range = (pd.to_datetime('2020-01-01').value // 10**9, pd.to_datetime('2020-07-01').value // 10**9) 
time_deltas = pd.timedelta_range(start='-5 days', end='5 days', freq='6H')

#Generate random MRN #s, admission dates, and encounters
#Generate between 3 and 10 encounters for each patient
mrns = np.random.randint(10000, 20000, 100)
admns = pd.to_datetime(np.random.randint(date_range[0], date_range[1], 100), unit = 's')
encs = [np.random.randint(10000, 99999, np.random.randint(3,10)) for mrn, admn in zip(mrns, admns)]
creats = np.clip(np.random.normal(loc = 1, scale = 0.3, size=100), a_min = 0, a_max = None)

#Combine the two dataframes
d1 = pd.DataFrame([mrns, admns, creats]).T
d2 = pd.DataFrame(encs)

d1.columns = ['mrn', 'admn', 'base_creat']
d2 = d2.add_prefix('enc_')

df = pd.concat([d1, d2], axis=1)
df = pd.melt(df, id_vars = d1.columns, value_name = 'enc').drop('variable', axis=1)

#Remove the duplicated & null values and reset the index
df = df[np.logical_and(~df.enc.isnull(), ~df.enc.duplicated())].reset_index(drop=True) 
print('Number of patients: {}\nNumber of admissions: {}\nNumber of encounters: {}'.format(df.mrn.unique().shape[0],
                                                                                          df.admn.unique().shape[0],
                                                                                          df.enc.unique().shape[0]))

df['creat'] = np.clip(df.base_creat + np.random.normal(loc = 0, scale = 0.25, size = df.shape[0]), a_min = 0.1, a_max = None)
df['time'] = df.admn + np.array([random.choice(time_deltas) for i in range(df.shape[0])])
df = df[['mrn', 'enc', 'admn', 'time', 'creat']]
df.head()

Number of patients: 100
Number of admissions: 100
Number of encounters: 606


,mrn,enc,admn,time,creat
0,12732,25741.0,2020-02-27 11:42:42,2020-02-29 05:42:42,1.11689
1,19845,81382.0,2020-05-12 06:02:54,2020-05-08 00:02:54,0.39054
2,13264,89464.0,2020-01-10 11:16:57,2020-01-12 17:16:57,1.35314
3,14859,75180.0,2020-06-07 12:27:38,2020-06-10 06:27:38,1.18235
4,19225,94917.0,2020-03-26 21:16:49,2020-03-28 09:16:49,1.23292


In [91]:
akiFlagger.returnAKIpatients(df)

AssertionError: 

In [61]:
df.creat.unique().shape

(606,)